JOEY

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np
import re
import pickle

In [ ]:
df = pd.read_csv('/Users/joey/Desktop/output4.csv')
df['Sent'] = pd.to_datetime(df['Sent'])
df = df[df['Sent']>datetime(1900, 1, 1, 0, 0, 0)]
df = df.reset_index(drop=True)
del df['Unnamed: 0.1']
df

In [ ]:
df = df[df['Sent']<datetime(2014, 1, 1, 0, 0, 0)]
df = df[df['Sent']>datetime(2013, 1, 1, 0, 0, 0)]


In [ ]:
# testing for name correction
a = 'Administration-Thomas Harrison <Abc.abc@gmail.com>'
b = 'Wiebler, Brian T.'
c = 'Brian T. Wiebler'
d = 'JT Burnette (jt@inkbridge.com)'
e = 'JTBurnette (jt@inkbridge.com)'
f = 'Ingram, M\'Lisa'
g = 'Mike V'
h = 'LCEM Mail'
i = 'Gary Yordon [mailto:gary@govinc.net]'
j = 'City Commission Office'
k = '(gary@govinc.net)<gary@govinc.net>'
l = 'alan1596@aol.com'
m = ' april salter'
re_name1 = re.compile(r'.*?([A-Z][a-z]+)\s?[A-Z]?\.?\s?([A-Z][a-z]+\s?[A-Z]?[a-z]+)')
re_name2 = re.compile(r'.*?([A-Z][a-z]+),\s?[A-Z]?\'?([A-Z][a-z]+)\s?[A-Z]?\.?')
re_name3 = re.compile(r'.*?([A-Z]+?)\s?([A-Z]?[a-z]+)\s?[\[\(]?.*\@.*[\)\]]?')
re_email = re.compile(r'.*?([a-zA-Z_]*\.?\w+@[a-zA-Z_]*\.?[a-zA-Z_]*\.?[a-zA-Z]{2,3})')

def standardize_name(string): # for now, only keep names
    if re_name1.match(string):
#         print(1)
        return re_name1.findall(string)[0][0]+' '+re_name1.findall(string)[0][1]
    elif re_name2.match(string):
#         print(2)   
        return re_name2.findall(string)[0][1].strip()+' '+re_name2.findall(string)[0][0]
    elif re_name3.match(string):
#         print(3)
        return re_name3.findall(string)[0][0].strip()+' '+re_name3.findall(string)[0][1]
    elif re_email.match(string.lower()):
#         print(4)
#         return re_email.findall(string.lower())[0].strip()
        return 
    else:
        return
#         return string.strip()
standardize_name(k)

In [ ]:
# convert pandas df to dictionary, only keep sender/recipient names and sent time
email_dic = {}
for idx, dataslice in enumerate(df.itertuples()):
    standardized_name = []
#   print(dataslice[4])
    for person in dataslice[4][2:-2].replace("'",'').strip().split(';'):
#         print(person)
        standardized_name.append(standardize_name(person))
#         print(standardize_name(person))
    for person in dataslice[5][2:-2].replace("'",'').strip().split(';'):
        if person != '':
            standardized_name.append(standardize_name(person))
    for name in standardized_name:
        if standardize_name(dataslice[2]) not in email_dic:
            email_dic[standardize_name(dataslice[2])] = []
        if standardize_name(dataslice[2]) != None:
            if name!=None:
                email_dic[standardize_name(dataslice[2])].append([name,dataslice[3]])
with open('email_dic.pickle', 'wb') as handle:
    pickle.dump(email_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# the key of the dictionary is sender's name, the value coresponds to the key is the recipient's name and sent time.
with open('email_dic.pickle', 'rb') as handle:
    email_dic = pickle.load(handle)
email_dic_c = email_dic.copy()

In [ ]:
all_names = []
for sender in email_dic_c.keys():
#     print(sender)
    if sender != None:
        all_names.append(sender)
    for e in email_dic_c[sender]:
        recipient = e[0]
        if recipient != None:
            all_names.append(recipient)


for idx in range(len(all_names)):
    all_names[idx] = all_names[idx].replace("'",'').strip()
    all_names[idx] = standardize_name(all_names[idx])
unique_names = list(set(all_names))
len(unique_names)

In [ ]:
unique_names.remove(None)
unique_names.sort()
unique_names[-100:]

In [ ]:
# create name to index dictionary and index to name dictionary for later use
name_id = {}
for idx,name in enumerate(unique_names):
    name_id[name] = idx
id_name = {}
for idx,name in enumerate(unique_names):
    id_name[idx] = name

In [ ]:
# create a square matrix to save number of emails sent and received by each person
sender_to_recipient = np.zeros((len(unique_names),len(unique_names)))
for sender in email_dic_c.keys():
    for e in email_dic_c[sender]:
        recipient = e[0]
        if sender in name_id and recipient in name_id:
            sender_to_recipient[name_id[sender],name_id[recipient]] = sender_to_recipient[name_id[sender],name_id[recipient]] +1
np.sum(sender_to_recipient)

In [ ]:
# plotting
G = nx.Graph()
edge_width = []
node_weight_sender = np.sum(sender_to_recipient,axis = 1)
node_weight_recipient = np.sum(sender_to_recipient,axis = 0)
node_weight_total = node_weight_sender + node_weight_recipient 
 
# node_weight is the size of node 
# the node weight has to be in a specific order(in the order of time when the node first added to the graph), 
# cannot just use node_weight_total
node_weight = [] 
for i in range(sender_to_recipient.shape[0]):
    for j in range(i,sender_to_recipient.shape[0]):
        # if there is more than 1 email between these 2 people, add node if haven't add. Add edge.
        if sender_to_recipient[i,j] + sender_to_recipient[j,i] > 1:
            if id_name[i] not in G.nodes():
                G.add_node(id_name[i])
                node_weight.append(node_weight_total[i])
            if id_name[j] not in G.nodes():
                    G.add_node(id_name[j])
                    node_weight.append(node_weight_total[j])
            G.add_edge(id_name[i], id_name[j],weight= 20/(sender_to_recipient[i,j] + sender_to_recipient[j,i] + 0.5*(node_weight_total[i]+ node_weight_total[j])))
            edge_width.append(sender_to_recipient[i,j] + sender_to_recipient[j,i])
print('done adding edges and nodes')

# find who should be labeled
node_have_label = {}
for i in range(sender_to_recipient.shape[0]):
    if node_weight_total[i]>250 and id_name[i] in G.nodes():
        node_have_label[id_name[i]] = id_name[i]

# edge_width is actrually edge strength. Bigger strength will lead to shorter distance
# edge_width = np.sqrt(np.array(edge_width))
edge_width = 0.2*(np.array(edge_width))
print('done adding labels')

plt.figure(figsize=(40,40))
# calculating node positions
pos = nx.spring_layout(G,iterations=30)
print('done calculating position')

nx.draw_networkx_nodes(G, pos, node_size= node_weight,node_color = 'black')
nx.draw_networkx_edges(G, pos, width= edge_width, edge_color = 'grey')
nx.draw_networkx_labels(G, pos, labels= node_have_label, font_size=24, font_color = 'red', font_family='sans-serif')

plt.axis('off')
plt.show()

In [ ]:
# test
edge_width.max()

In [ ]:
# test
G = nx.Graph()

G.add_edge('d','a',weight = 0.1)
G.add_edge('d','b',weight = 100)
G.add_edge('c','a',weight = 0.1)

pos = nx.spring_layout(G)

nx.draw_networkx_nodes(G, pos,node_color = 'black')
nx.draw_networkx_edges(G, pos, width=1, edge_color = 'grey')
nx.draw_networkx_labels(G, pos, labels={'a':'a','d':'d'}, font_size=30, font_color='blue',font_family='sans-serif')

# old codes

In [ ]:
# # for i in range(5):
# i = 1
# df_for_plot2 = df_for_plot[df_for_plot['Sent']<datetime(2013+i, 1, 1, 0, 0, 0)]
# df_for_plot3 = df_for_plot2[df_for_plot2['Sent']>datetime(2012+i, 1, 1, 0, 0, 0)]

# G = nx.from_pandas_edgelist(df_for_plot3, 'From','To')
# count = df_for_plot3['From'].append(df_for_plot3['To']).value_counts()
# nodesizes = np.zeros(len(list(G.nodes)))
# for i in range(len(list(G.nodes))):
#     if list(G.nodes)[i] == None:
#         nodesizes[i] = 0
#     else:
#         nodesizes[i] = count[count.index == list(G.nodes)[i]][0]
# index = nodesizes.argsort()[-10:][::-1]
# Top10 = [list(G.nodes)[i] for i in index]
# Top10_val = [nodesizes[i] for i in index]
# print(list(zip(Top10, Top10_val)))

# plt.figure(figsize = (40,30))
# pos = nx.spring_layout(G, k = 0.1, iterations = 30)
# nx.draw_networkx(G, pos, node_size = nodesizes, node_color = 'black', with_labels = True, edge_color='grey')
# plt.axis('off')
# plt.show()

In [ ]:
# l1 = list(G.nodes())
# l2 = big_names
# l3 = [x for x in l1 if x not in l2]
# G.remove_nodes_from(l3)

In [ ]:
# count = df_for_plot3['From'].append(df_for_plot3['To']).value_counts()
# nodesizes = np.zeros(len(list(G.nodes)))
# for i in range(len(list(G.nodes))):
#     if list(G.nodes)[i] == None:
#         nodesizes[i] = 0
#     else:
#         nodesizes[i] = count[count.index == list(G.nodes)[i]][0]
# index = nodesizes.argsort()[-10:][::-1]
# Top10 = [list(G.nodes)[i] for i in index]
# Top10_val = [nodesizes[i] for i in index]
# print(list(zip(Top10, Top10_val)))

# plt.figure(figsize = (40,30))
# pos = nx.spring_layout(G, k = 0.1, iterations = 10)
# nx.draw_networkx(G, pos, node_size = nodesizes, node_color = 'black', with_labels = True, edge_color='grey')
# plt.axis('off')
# plt.savefig('nx_200.png')
# plt.show()